In [5]:
import numpy as np
import sys
from scipy.interpolate import UnivariateSpline
import cv2
from tqdm import tqdm

sys.path.append('../..')

from core.detections import load_video_detections

video_path = "/home/luis/workspace/vacocam_render/source/prerender.mp4"

_, detections = load_video_detections(video_path, module="track", version="vacocam")
chunk_size = 64
overlap_size = 16

X_interpolated = []
Y_interpolated = []

current_index = 0

while current_index < len(detections):
    current_overlap = overlap_size if current_index > 0 else 0

    x = X_interpolated[len(X_interpolated) - current_overlap:]
    y = Y_interpolated[len(Y_interpolated) - current_overlap:]
    weights = [1.] * current_overlap

    last_center = None if current_index == 0 else (X_interpolated[-1], Y_interpolated[-1])

    detections_start = current_index
    detections_end = min(current_index + chunk_size - 1, len(detections))

    no_detection = True

    while no_detection or ((detections_end - detections_start) < chunk_size):
        if detections_end >= len(detections):
            break

        frame_detection = detections[detections_end]

        if len(frame_detection) > 0:
            no_detection = False

        detections_end += 1

    detections_chunk = detections[detections_start:detections_end]

    for frame_detections in detections_chunk:
        if len(frame_detections) > 0:
            min_distance = float("inf")

            for bbox in frame_detections.xyxy:
                center = (bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2

                distance = 0 if last_center is None else np.linalg.norm(np.array(center) - np.array(last_center))
                distance = float(distance)

                if distance < min_distance:
                    min_distance = distance
                    last_center = center
            
            x.append(last_center[0])
            y.append(last_center[1])

            weights.append(1)
        else:
            x.append(1920 / 2)
            y.append(1080 / 2)
            weights.append(1e-10)

    # spl_x = UnivariateSpline(range(len(x)), x, w=weights, k=3, s=len(weights) * 32)
    # spl_y = UnivariateSpline(range(len(y)), y, w=weights, k=3, s=len(weights) * 32)
    # spl_x = UnivariateSpline(range(len(x)), x, w=weights, k=2, s=len(weights) * 32)
    # spl_y = UnivariateSpline(range(len(y)), y, w=weights, k=2, s=len(weights) * 32)

    spl_x = UnivariateSpline(range(len(x)), x, w=weights, k=1)
    spl_y = UnivariateSpline(range(len(y)), y, w=weights, k=1)

    # Generate more points for a smoother curve
    interpolated_x = spl_x(range(len(x)))
    interpolated_y = spl_y(range(len(y)))

    X_interpolated.extend(interpolated_x[current_overlap:])
    Y_interpolated.extend(interpolated_y[current_overlap:])

    current_index += len(detections_chunk)

print(f"Interpolated {len(X_interpolated)} points")
print(f"Original {len(detections)} points")
def write_video(video_path, X_interpolated, Y_interpolated):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('robust_spline.mp4', fourcc, 30.0, (1920, 1080))

    video = cv2.VideoCapture(video_path)

    for i in tqdm(range(len(X_interpolated))):
        ret, frame = video.read()
        if not ret:
            break

        cv2.circle(frame, (int(X_interpolated[i]), int(Y_interpolated[i])), 10, (0, 0, 255), -1)

        out.write(frame)

    out.release()

loading detections from /home/luis/workspace/vacocam_render/source/track/prerender_detections_vacocam.npy
loaded 112300 detections from /home/luis/workspace/vacocam_render/source/track/prerender_detections_vacocam.npy


/home/luis/workspace/vacocam_render/venv/lib/python3.10/site-packages/scipy/interpolate/_fitpack2.py:313: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


Interpolated 112300 points
Original 112300 points
